In [1]:
# Setting up and starting the interactive session
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
 
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 6e2ccdf6-b73f-40f8-adfa-4425fd81383b
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session 6e2ccdf6-b73f-40f8-adfa-4425fd81383b to get into ready status...
Session 6e2ccdf6-b73f-40f8-adfa-4425fd81383b ha

In [2]:
# Reading from the customers table in the glue data catalog using a dynamic frame
dynamicFrameCustomers = glueContext.create_dynamic_frame.from_catalog(database = "mygludatabse", table_name = "customers")

# Showing the top 10 rows from the dynamic dataframe
dynamicFrameCustomers.show(10)

# Since, the above python code will show only top nine records as Customerid, Firstname, Lastname and Fullname as first row. 
# So, by applying mapping to rename the columns for running the upcoming codes as Customerid, Firstname, Lastname and Fullname as headers.
mapped_dynamic_frame = dynamicFrameCustomers.apply_mapping([
    ('col0', 'string', 'customerid', 'string'),
    ('col1', 'string', 'firstname', 'string'),
    ('col2', 'string', 'lastname', 'string'),
    ('col3', 'string', 'fullname', 'string')
])

# Converting the DynamicFrame to an iterable of records
records = mapped_dynamic_frame.toDF().collect()

# Shifting the rows (make the second row the first row)
header = records.pop(0)

# Creating a new DynamicFrame from the remaining records
newdynamicFrameCustomers = DynamicFrame.fromDF(spark.createDataFrame(records, schema=header), glueContext, "newdynamicFrameCustomers")

# Showing the DataFrame with the new column names to verify
newdynamicFrameCustomers.show(10)

{"col0": "CustomerID", "col1": "Firstname", "col2": "Lastname", "col3": "Fullname"}
{"col0": "293", "col1": "Catherine", "col2": "Abel", "col3": "Catherine Abel"}
{"col0": "295", "col1": "Kim", "col2": "Abercrombie", "col3": "Kim Abercrombie"}
{"col0": "297", "col1": "Humberto", "col2": "Acevedo", "col3": "Humberto Acevedo"}
{"col0": "291", "col1": "Gustavo", "col2": "Achong", "col3": "Gustavo Achong"}
{"col0": "299", "col1": "Pilar", "col2": "Ackerman", "col3": "Pilar Ackerman"}
{"col0": "305", "col1": "Carla", "col2": "Adams", "col3": "Carla Adams"}
{"col0": "301", "col1": "Frances", "col2": "Adams", "col3": "Frances Adams"}
{"col0": "307", "col1": "Jay", "col2": "Adams", "col3": "Jay Adams"}
{"col0": "309", "col1": "Ronald", "col2": "Adina", "col3": "Ronald Adina"}
{"CustomerID": "293", "Firstname": "Catherine", "Lastname": "Abel", "Fullname": "Catherine Abel"}
{"CustomerID": "295", "Firstname": "Kim", "Lastname": "Abercrombie", "Fullname": "Kim Abercrombie"}
{"CustomerID": "297", "

In [3]:
# Checking types in dynamic frame
newdynamicFrameCustomers.printSchema()

root
|-- CustomerID: string
|-- Firstname: string
|-- Lastname: string
|-- Fullname: string


In [4]:
# Counting The Number of Rows in a Dynamic Dataframe
newdynamicFrameCustomers.count()

636


In [6]:
# Selecting certain fields from a Dynamic DataFrame
dyfCustomerSelectFields = newdynamicFrameCustomers.select_fields(["CustomerID", "Fullname"])
# Showing top 10
dyfCustomerSelectFields.show(10)

{"CustomerID": "293", "Fullname": "Catherine Abel"}
{"CustomerID": "295", "Fullname": "Kim Abercrombie"}
{"CustomerID": "297", "Fullname": "Humberto Acevedo"}
{"CustomerID": "291", "Fullname": "Gustavo Achong"}
{"CustomerID": "299", "Fullname": "Pilar Ackerman"}
{"CustomerID": "305", "Fullname": "Carla Adams"}
{"CustomerID": "301", "Fullname": "Frances Adams"}
{"CustomerID": "307", "Fullname": "Jay Adams"}
{"CustomerID": "309", "Fullname": "Ronald Adina"}
{"CustomerID": "311", "Fullname": "Samuel Agcaoili"}


In [7]:
# Dropping Fields of Dynamic Frame
dyfCustomerDropFields = newdynamicFrameCustomers.drop_fields(["Firstname","Lastname"])
# Showing Top 10 rows of dyfCustomerDropFields Dynamic Frame
dyfCustomerDropFields.show(10)

{"CustomerID": "293", "Fullname": "Catherine Abel"}
{"CustomerID": "295", "Fullname": "Kim Abercrombie"}
{"CustomerID": "297", "Fullname": "Humberto Acevedo"}
{"CustomerID": "291", "Fullname": "Gustavo Achong"}
{"CustomerID": "299", "Fullname": "Pilar Ackerman"}
{"CustomerID": "305", "Fullname": "Carla Adams"}
{"CustomerID": "301", "Fullname": "Frances Adams"}
{"CustomerID": "307", "Fullname": "Jay Adams"}
{"CustomerID": "309", "Fullname": "Ronald Adina"}
{"CustomerID": "311", "Fullname": "Samuel Agcaoili"}


In [8]:
# Mapping array for column rename fullname -> name
mapping=[("Customerid", "string", "Customerid","string"),("Fullname", "string", "Name", "string")]
# Applying the mapping to rename fullname -> name
dfyMapping = ApplyMapping.apply(frame = dyfCustomerDropFields, mappings = mapping, transformation_ctx = "applymapping1")

# Showing the new dynamic frame with name column
dfyMapping.show(10)

{"Customerid": "293", "Name": "Catherine Abel"}
{"Customerid": "295", "Name": "Kim Abercrombie"}
{"Customerid": "297", "Name": "Humberto Acevedo"}
{"Customerid": "291", "Name": "Gustavo Achong"}
{"Customerid": "299", "Name": "Pilar Ackerman"}
{"Customerid": "305", "Name": "Carla Adams"}
{"Customerid": "301", "Name": "Frances Adams"}
{"Customerid": "307", "Name": "Jay Adams"}
{"Customerid": "309", "Name": "Ronald Adina"}
{"Customerid": "311", "Name": "Samuel Agcaoili"}


In [9]:
# Filtering dynamicFrameCustomers for customers who have the last name Adams
dyfFilter= Filter.apply(frame = newdynamicFrameCustomers,
f = lambda x: x["Lastname"] in "Adams"
)
# Showing the top 10 customers from the filtered Dynamic frame
dyfFilter.show(10)

{"Lastname": "Adams", "CustomerID": "305", "Firstname": "Carla", "Fullname": "Carla Adams"}
{"Lastname": "Adams", "CustomerID": "301", "Firstname": "Frances", "Fullname": "Frances Adams"}
{"Lastname": "Adams", "CustomerID": "307", "Firstname": "Jay", "Fullname": "Jay Adams"}


In [10]:
# Reading from the customers table in the glue data catalog using a dynamic frame
dynamicFrameOrders = glueContext.create_dynamic_frame.from_catalog(
database = "mygludatabse",
table_name = "orders"
)
# Applying mapping to rename the columns for orders
mapped_dynamic_frame_orders = dynamicFrameOrders.apply_mapping([
    ('col0', 'string', 'SalesOrderID', 'string'),
    ('col1', 'string', 'SalesOrderDetailID', 'string'),
    ('col2', 'string', 'OrderDate', 'string'),
    ('col3', 'string', 'DueDate', 'string'),
    ('col4', 'string', 'ShipDate', 'string'),
    ('col5', 'string', 'EmployeeID', 'string'),
    ('col6', 'string', 'CustomerID', 'string'),
    ('col7', 'string', 'SubTotal', 'string'),
    ('col8', 'string', 'TaxAmt', 'string'),
    ('col9', 'string', 'Freight', 'string'),
    ('col10', 'string', 'TotalDue', 'string'),
    ('col11', 'string', 'ProductID', 'string'),
    ('col12', 'string', 'OrderQty', 'string'),
    ('col13', 'string', 'UnitPrice', 'string'),
    ('col14', 'string', 'UnitPriceDiscount', 'string'),
    ('col15', 'string', 'LineTotal', 'string')
])

# Converting the DynamicFrame to an iterable of records
records = mapped_dynamic_frame_orders.toDF().collect()

# Shifting the rows (make the second row the first row)
header = records.pop(0)

# Creating a new DynamicFrame from the remaining records
newdynamicFrameOrders = DynamicFrame.fromDF(spark.createDataFrame(records, schema=header), glueContext, "newdynamicFrameOrders")

# Showing the DataFrame with the new column names to verify
newdynamicFrameOrders.show(10)

{"SalesOrderID": "43659", "SalesOrderDetailID": "1", "OrderDate": "5/31/2011", "DueDate": "6/12/2011", "ShipDate": "6/7/2011", "EmployeeID": "279", "CustomerID": "1045", "SubTotal": "20565.6206", "TaxAmt": "1971.5149", "Freight": "616.0984", "TotalDue": "23153.2339", "ProductID": "776", "OrderQty": "1", "UnitPrice": "2024.994", "UnitPriceDiscount": "0", "LineTotal": "2024.994"}
{"SalesOrderID": "43659", "SalesOrderDetailID": "2", "OrderDate": "5/31/2011", "DueDate": "6/12/2011", "ShipDate": "6/7/2011", "EmployeeID": "279", "CustomerID": "1045", "SubTotal": "20565.6206", "TaxAmt": "1971.5149", "Freight": "616.0984", "TotalDue": "23153.2339", "ProductID": "777", "OrderQty": "3", "UnitPrice": "2024.994", "UnitPriceDiscount": "0", "LineTotal": "6074.982"}
{"SalesOrderID": "43659", "SalesOrderDetailID": "3", "OrderDate": "5/31/2011", "DueDate": "6/12/2011", "ShipDate": "6/7/2011", "EmployeeID": "279", "CustomerID": "1045", "SubTotal": "20565.6206", "TaxAmt": "1971.5149", "Freight": "616.098

In [11]:
# Joining two dynamic frames on an equality join
dyfjoin = newdynamicFrameCustomers.join(["CustomerID"],["CustomerID"],newdynamicFrameOrders)
# Showing top 10 rows for the joined dynamic
dyfjoin.show(10)

{"DueDate": "12/13/2011", "Firstname": "Susan", "ProductID": "770", "SubTotal": "5488.8419", "CustomerID": "907", "Lastname": "French", "TaxAmt": "526.9288", "TotalDue": "6180.436", "Fullname": "Susan French", "LineTotal": "419.4589", "ShipDate": "12/8/2011", "EmployeeID": "278", "SalesOrderDetailID": "5331", "Freight": "164.6653", "UnitPriceDiscount": "0", ".CustomerID": "907", "OrderDate": "12/1/2011", "UnitPrice": "419.4589", "OrderQty": "1", "SalesOrderID": "45058"}
{"DueDate": "12/13/2011", "Firstname": "Susan", "ProductID": "760", "SubTotal": "5488.8419", "CustomerID": "907", "Lastname": "French", "TaxAmt": "526.9288", "TotalDue": "6180.436", "Fullname": "Susan French", "LineTotal": "419.4589", "ShipDate": "12/8/2011", "EmployeeID": "278", "SalesOrderDetailID": "5332", "Freight": "164.6653", "UnitPriceDiscount": "0", ".CustomerID": "907", "OrderDate": "12/1/2011", "UnitPrice": "419.4589", "OrderQty": "1", "SalesOrderID": "45058"}
{"DueDate": "12/13/2011", "Firstname": "Susan", "P

In [15]:
# Writing down the data in a Dynamic Frame to S3 location
output_path = "s3://myawsgluebkt/ write_down_dyf_to_s3/"

glueContext.write_dynamic_frame.from_options(
    frame=dynamicFrameCustomers,
    connection_type="s3",
    connection_options={"path": output_path},
    format="csv",
    format_options={"separator": ","},
    transformation_ctx="datasink2"
)

In [26]:
# Writing data from dynamicFrameCustomers to customers_write_dyf table using the metadata stored in the Glue Data Catalog
glueContext.write_dynamic_frame.from_catalog(
    frame=newdynamicFrameCustomers,
    database="mygludatabse",
    table_name="customers_write_dyf"
)

In [27]:
# Dynamic Frame to Spark DataFrame
sparkDf = newdynamicFrameCustomers.toDF()
# Showing spark DF
sparkDf.show(10)

+----------+---------+-----------+----------------+
|CustomerID|Firstname|   Lastname|        Fullname|
+----------+---------+-----------+----------------+
|       293|Catherine|       Abel|  Catherine Abel|
|       295|      Kim|Abercrombie| Kim Abercrombie|
|       297| Humberto|    Acevedo|Humberto Acevedo|
|       291|  Gustavo|     Achong|  Gustavo Achong|
|       299|    Pilar|   Ackerman|  Pilar Ackerman|
|       305|    Carla|      Adams|     Carla Adams|
|       301|  Frances|      Adams|   Frances Adams|
|       307|      Jay|      Adams|       Jay Adams|
|       309|   Ronald|      Adina|    Ronald Adina|
|       311|   Samuel|   Agcaoili| Samuel Agcaoili|
+----------+---------+-----------+----------------+
only showing top 10 rows


In [28]:
# Selecting columns from spark dataframe
dfSelect = sparkDf.select("Customerid","Fullname")
# Showing selected
dfSelect.show(10)

+----------+----------------+
|Customerid|        Fullname|
+----------+----------------+
|       293|  Catherine Abel|
|       295| Kim Abercrombie|
|       297|Humberto Acevedo|
|       291|  Gustavo Achong|
|       299|  Pilar Ackerman|
|       305|     Carla Adams|
|       301|   Frances Adams|
|       307|       Jay Adams|
|       309|    Ronald Adina|
|       311| Samuel Agcaoili|
+----------+----------------+
only showing top 10 rows


In [29]:
# Importing lit from sql functions
from pyspark.sql.functions import lit
# Adding new column to spark dataframe
dfNewColumn = sparkDf.withColumn("Date", lit("2022-07-24"))
# Showing df with new column
dfNewColumn.show(10)

+----------+---------+-----------+----------------+----------+
|CustomerID|Firstname|   Lastname|        Fullname|      Date|
+----------+---------+-----------+----------------+----------+
|       293|Catherine|       Abel|  Catherine Abel|2022-07-24|
|       295|      Kim|Abercrombie| Kim Abercrombie|2022-07-24|
|       297| Humberto|    Acevedo|Humberto Acevedo|2022-07-24|
|       291|  Gustavo|     Achong|  Gustavo Achong|2022-07-24|
|       299|    Pilar|   Ackerman|  Pilar Ackerman|2022-07-24|
|       305|    Carla|      Adams|     Carla Adams|2022-07-24|
|       301|  Frances|      Adams|   Frances Adams|2022-07-24|
|       307|      Jay|      Adams|       Jay Adams|2022-07-24|
|       309|   Ronald|      Adina|    Ronald Adina|2022-07-24|
|       311|   Samuel|   Agcaoili| Samuel Agcaoili|2022-07-24|
+----------+---------+-----------+----------------+----------+
only showing top 10 rows


In [30]:
# Importing concat from functions
from pyspark.sql.functions import concat
# Creating another full name column
dfNewFullName = sparkDf.withColumn("New_full_name",concat("Firstname",concat(lit(' '),"Lastname")))
# Showing full name column
dfNewFullName.show(10)

+----------+---------+-----------+----------------+----------------+
|CustomerID|Firstname|   Lastname|        Fullname|   New_full_name|
+----------+---------+-----------+----------------+----------------+
|       293|Catherine|       Abel|  Catherine Abel|  Catherine Abel|
|       295|      Kim|Abercrombie| Kim Abercrombie| Kim Abercrombie|
|       297| Humberto|    Acevedo|Humberto Acevedo|Humberto Acevedo|
|       291|  Gustavo|     Achong|  Gustavo Achong|  Gustavo Achong|
|       299|    Pilar|   Ackerman|  Pilar Ackerman|  Pilar Ackerman|
|       305|    Carla|      Adams|     Carla Adams|     Carla Adams|
|       301|  Frances|      Adams|   Frances Adams|   Frances Adams|
|       307|      Jay|      Adams|       Jay Adams|       Jay Adams|
|       309|   Ronald|      Adina|    Ronald Adina|    Ronald Adina|
|       311|   Samuel|   Agcaoili| Samuel Agcaoili| Samuel Agcaoili|
+----------+---------+-----------+----------------+----------------+
only showing top 10 rows


In [31]:
# Dropping column from spark dataframe
dfDropCol = sparkDf.drop("Firstname","Lastname")
# Showing dropped column df
dfDropCol.show(10)

+----------+----------------+
|CustomerID|        Fullname|
+----------+----------------+
|       293|  Catherine Abel|
|       295| Kim Abercrombie|
|       297|Humberto Acevedo|
|       291|  Gustavo Achong|
|       299|  Pilar Ackerman|
|       305|     Carla Adams|
|       301|   Frances Adams|
|       307|       Jay Adams|
|       309|    Ronald Adina|
|       311| Samuel Agcaoili|
+----------+----------------+
only showing top 10 rows


In [32]:
# Renaming column in Spark dataframe
dfRenameCol = sparkDf.withColumnRenamed("Fullname","Full_name")
# Showing renamed column dataframe
dfRenameCol.show(10)

+----------+---------+-----------+----------------+
|CustomerID|Firstname|   Lastname|       Full_name|
+----------+---------+-----------+----------------+
|       293|Catherine|       Abel|  Catherine Abel|
|       295|      Kim|Abercrombie| Kim Abercrombie|
|       297| Humberto|    Acevedo|Humberto Acevedo|
|       291|  Gustavo|     Achong|  Gustavo Achong|
|       299|    Pilar|   Ackerman|  Pilar Ackerman|
|       305|    Carla|      Adams|     Carla Adams|
|       301|  Frances|      Adams|   Frances Adams|
|       307|      Jay|      Adams|       Jay Adams|
|       309|   Ronald|      Adina|    Ronald Adina|
|       311|   Samuel|   Agcaoili| Samuel Agcaoili|
+----------+---------+-----------+----------------+
only showing top 10 rows


In [33]:
# Groupping by lastname then print counts of lastname and show
sparkDf.groupBy("Lastname").count().show(10)


+--------+-----+
|Lastname|count|
+--------+-----+
|  Bailey|    1|
|Atkinson|    1|
|  Avalos|    1|
| Bacalzo|    1|
|Bennetts|    1|
|Barnhill|    1|
|   Black|    1|
|   Ayers|    1|
|  Banack|    1|
|  Becker|    1|
+--------+-----+
only showing top 10 rows


In [34]:
# Filtering spark DataFrame for customers who have the last name Adams
sparkDf.filter(sparkDf["Lastname"] == "Adams").show(10)

+----------+---------+--------+-------------+
|CustomerID|Firstname|Lastname|     Fullname|
+----------+---------+--------+-------------+
|       305|    Carla|   Adams|  Carla Adams|
|       301|  Frances|   Adams|Frances Adams|
|       307|      Jay|   Adams|    Jay Adams|
+----------+---------+--------+-------------+


In [62]:
# Where clause spark DataFrame for customers who have the last name Adams
sparkDf.where("Lastname =='Adams'").show(10)

+----------+---------+--------+-------------+
|Customerid|Firstname|Lastname|     Fullname|
+----------+---------+--------+-------------+
|       305|    Carla|   Adams|  Carla Adams|
|       301|  Frances|   Adams|Frances Adams|
|       307|      Jay|   Adams|    Jay Adams|
+----------+---------+--------+-------------+


In [35]:
# Converting Orders table from Dynamic Frame to Spark DataFrame
sparkDf1 = newdynamicFrameOrders.toDF()
# Showing spark DF
sparkDf1.show(10)

+------------+------------------+---------+---------+--------+----------+----------+----------+---------+--------+----------+---------+--------+---------+-----------------+---------+
|SalesOrderID|SalesOrderDetailID|OrderDate|  DueDate|ShipDate|EmployeeID|CustomerID|  SubTotal|   TaxAmt| Freight|  TotalDue|ProductID|OrderQty|UnitPrice|UnitPriceDiscount|LineTotal|
+------------+------------------+---------+---------+--------+----------+----------+----------+---------+--------+----------+---------+--------+---------+-----------------+---------+
|       43659|                 1|5/31/2011|6/12/2011|6/7/2011|       279|      1045|20565.6206|1971.5149|616.0984|23153.2339|      776|       1| 2024.994|                0| 2024.994|
|       43659|                 2|5/31/2011|6/12/2011|6/7/2011|       279|      1045|20565.6206|1971.5149|616.0984|23153.2339|      777|       3| 2024.994|                0| 6074.982|
|       43659|                 3|5/31/2011|6/12/2011|6/7/2011|       279|      1045|2

In [38]:
# Inner Join Customers Spark DF to Orders Spark DF
innerJoinDf = sparkDf.join(sparkDf1, sparkDf['CustomerID'] == sparkDf1['CustomerID'], "inner")
innerJoinDf.show(10, truncate=False)

+----------+---------+--------+------------+------------+------------------+---------+---------+--------+----------+----------+---------+--------+--------+---------+---------+--------+---------+-----------------+---------+
|CustomerID|Firstname|Lastname|Fullname    |SalesOrderID|SalesOrderDetailID|OrderDate|DueDate  |ShipDate|EmployeeID|CustomerID|SubTotal |TaxAmt  |Freight |TotalDue |ProductID|OrderQty|UnitPrice|UnitPriceDiscount|LineTotal|
+----------+---------+--------+------------+------------+------------------+---------+---------+--------+----------+----------+---------+--------+--------+---------+---------+--------+---------+-----------------+---------+
|487       |Mae      |Black   |Mae Black   |43670       |111               |5/31/2011|6/12/2011|6/7/2011|275       |487       |6122.082 |587.5603|183.6126|6893.2549|710      |1       |5.7      |0                |5.7      |
|487       |Mae      |Black   |Mae Black   |43670       |112               |5/31/2011|6/12/2011|6/7/2011|275

In [39]:
# Getting customers that only have surname Adams
dfAdams = sparkDf.filter(sparkDf['Lastname'] == 'Adams')
dfAdams.show(10)

+----------+---------+--------+-------------+
|CustomerID|Firstname|Lastname|     Fullname|
+----------+---------+--------+-------------+
|       305|    Carla|   Adams|  Carla Adams|
|       301|  Frances|   Adams|Frances Adams|
|       307|      Jay|   Adams|    Jay Adams|
+----------+---------+--------+-------------+


In [40]:
# inner join on Adams DF and orders
dfAdams.join(sparkDf1,dfAdams.CustomerID == sparkDf1.CustomerID,"inner").show(10)

+----------+---------+--------+-------------+------------+------------------+---------+---------+--------+----------+----------+----------+---------+---------+----------+---------+--------+---------+-----------------+---------+
|CustomerID|Firstname|Lastname|     Fullname|SalesOrderID|SalesOrderDetailID|OrderDate|  DueDate|ShipDate|EmployeeID|CustomerID|  SubTotal|   TaxAmt|  Freight|  TotalDue|ProductID|OrderQty|UnitPrice|UnitPriceDiscount|LineTotal|
+----------+---------+--------+-------------+------------+------------------+---------+---------+--------+----------+----------+----------+---------+---------+----------+---------+--------+---------+-----------------+---------+
|       301|  Frances|   Adams|Frances Adams|       46634|             11379|5/30/2012|6/11/2012|6/6/2012|       281|       301|  733.2159|  87.2586|  27.2683|  847.7428|      849|       1|   35.994|                0|   35.994|
|       301|  Frances|   Adams|Frances Adams|       46634|             11380|5/30/2012|6

In [41]:
#left join on orders and adams df
sparkDf1.join(dfAdams,dfAdams.CustomerID == sparkDf1.CustomerID,"left").show(10)

+------------+------------------+---------+----------+---------+----------+----------+----------+---------+---------+----------+---------+--------+---------+-----------------+---------+----------+---------+--------+--------+
|SalesOrderID|SalesOrderDetailID|OrderDate|   DueDate| ShipDate|EmployeeID|CustomerID|  SubTotal|   TaxAmt|  Freight|  TotalDue|ProductID|OrderQty|UnitPrice|UnitPriceDiscount|LineTotal|CustomerID|Firstname|Lastname|Fullname|
+------------+------------------+---------+----------+---------+----------+----------+----------+---------+---------+----------+---------+--------+---------+-----------------+---------+----------+---------+--------+--------+
|       44552|              3682|10/1/2011|10/13/2011|10/8/2011|       279|       339|97915.4075|9418.0316|2943.1349|110276.574|      715|       2|  28.8404|                0|  57.6808|      null|     null|    null|    null|
|       44552|              3683|10/1/2011|10/13/2011|10/8/2011|       279|       339|97915.4075|941

In [42]:
# Importing Dynamic DataFrame class
from awsglue.dynamicframe import DynamicFrame
# Converting from Spark Data Frame to Glue Dynamic Frame
dyfCustomersConvert = DynamicFrame.fromDF(sparkDf, glueContext, "convert")
# Showing converted Glue Dynamic Frame
dyfCustomersConvert.show(10)

{"CustomerID": "293", "Firstname": "Catherine", "Lastname": "Abel", "Fullname": "Catherine Abel"}
{"CustomerID": "295", "Firstname": "Kim", "Lastname": "Abercrombie", "Fullname": "Kim Abercrombie"}
{"CustomerID": "297", "Firstname": "Humberto", "Lastname": "Acevedo", "Fullname": "Humberto Acevedo"}
{"CustomerID": "291", "Firstname": "Gustavo", "Lastname": "Achong", "Fullname": "Gustavo Achong"}
{"CustomerID": "299", "Firstname": "Pilar", "Lastname": "Ackerman", "Fullname": "Pilar Ackerman"}
{"CustomerID": "305", "Firstname": "Carla", "Lastname": "Adams", "Fullname": "Carla Adams"}
{"CustomerID": "301", "Firstname": "Frances", "Lastname": "Adams", "Fullname": "Frances Adams"}
{"CustomerID": "307", "Firstname": "Jay", "Lastname": "Adams", "Fullname": "Jay Adams"}
{"CustomerID": "309", "Firstname": "Ronald", "Lastname": "Adina", "Fullname": "Ronald Adina"}
{"CustomerID": "311", "Firstname": "Samuel", "Lastname": "Agcaoili", "Fullname": "Samuel Agcaoili"}


In [43]:
# Writing down the data in converted Dynamic Frame to S3 location.
output_path = "s3://myawsgluebkt/write_down_dyf_to_s3/" 
glueContext.write_dynamic_frame.from_options(
    frame=dyfCustomersConvert,
    connection_type="s3",
    connection_options={"path": output_path},
    format="csv",
    format_options={
        "separator": ","
    },
    transformation_ctx="datasink2"
)

In [44]:
# Writing data from the converted to customers_write_dyf table using the metadata stored in the glue data catalog
glueContext.write_dynamic_frame.from_catalog(
frame = dyfCustomersConvert,
database = "mygludatabse", table_name = "customers_write_dyf")